In [6]:
import os
import pandas as pd
import datetime
import re

# Constants
DELAY_THRESHOLD = 15  # minutes
LAST_DAYS = 7  # number of days to filter

# Function to check if a string is numeric
def is_numeric(s):
    return s.replace('+', '', 1).isdigit()

# Function to filter messages from the last 7 days
def filter_last_7_days_messages(file_path):
    print(f"Filtering messages from the last 7 days in file: {file_path}")  # Debug
    recent_messages = []
    current_date = datetime.datetime.now()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) -', line)
            if match:
                timestamp = datetime.datetime.strptime(match.group(1), '%d/%m/%y, %I:%M %p')
                if (current_date - timestamp).days < LAST_DAYS:
                    recent_messages.append(line.strip())
    print(f"Found {len(recent_messages)} messages from the last 7 days.")  # Debug
    return recent_messages

# Function to calculate delays and identify the person
def calculate_delays_and_identify_person(messages):
    print("Calculating delays and identifying the person...")  # Debug
    delays = []
    last_person_message_time = None
    for message in messages:
        timestamp_str, sender_and_message = message.split(' - ', 1)
        timestamp = datetime.datetime.strptime(timestamp_str, '%d/%m/%y, %I:%M %p')
        sender = sender_and_message.split(':', 1)[0]
        if not is_numeric(sender):  # Person's message
            if last_person_message_time and (timestamp - last_person_message_time).total_seconds() / 60 > DELAY_THRESHOLD:
                delays.append(last_person_message_time.strftime('%d/%m/%y, %I:%M %p'))
            last_person_message_time = timestamp
    print(f"Delays identified: {delays}")  # Debug
    return delays

# Main analysis process
def main_analysis(root_directory):
    print(f"Starting delay analysis in root directory: {root_directory}")  # Debug
    analysis_table = pd.DataFrame(columns=['Date', 'Chat Group Name', 'Person', 'Time of Delay', 'Last 7 Messages'])
    
    # Iterate over the directory structure
    for date_folder in os.listdir(root_directory):
        date_path = os.path.join(root_directory, date_folder)
        print(f"Processing date folder: {date_folder}")  # Debug
        for team_folder in os.listdir(date_path):
            team_path = os.path.join(date_path, team_folder)
            print(f"Processing team folder: {team_folder}")  # Debug
            for person_folder in os.listdir(team_path):
                person_path = os.path.join(team_path, person_folder)
                print(f"Processing person folder: {person_folder}")  # Debug
                for file in os.listdir(person_path):
                    if file.endswith('.txt'):
                        chat_file_path = os.path.join(person_path, file)
                        messages = filter_last_7_days_messages(chat_file_path)
                        delays = calculate_delays_and_identify_person(messages)
                        last_7_messages = ' | '.join(messages[-7:])
                        for delay in delays:
                            analysis_table = analysis_table.append({
                                'Date': date_folder, 
                                'Chat Group Name': team_folder, 
                                'Person': person_folder, 
                                'Time of Delay': delay, 
                                'Last 7 Messages': last_7_messages
                            }, ignore_index=True)
                            print(f"Added delay entry for {person_folder} on {date_folder}")  # Debug

    print("Analysis complete. Returning DataFrame.")  # Debug
    return analysis_table

# Run the analysis
root_directory = 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\V1_maurice\\TestingData'  # Replace with the actual path
delay_analysis_table = main_analysis(root_directory)
print(delay_analysis_table.head())  # Displaying the first few rows of the analysis table


Starting delay analysis in root directory: C:\Users\mauriceyeng\Python\Daily-Reports\Test\V1_maurice\TestingData
Processing date folder: 2023-11-18
Processing team folder: EWYL TEAM
Processing person folder: Ananya_Edoofa
Filtering messages from the last 7 days in file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\V1_maurice\TestingData\2023-11-18\EWYL TEAM\Ananya_Edoofa\WhatsApp Chat with Twazanga EWYL22E1057.txt
Found 0 messages from the last 7 days.
Calculating delays and identifying the person...
Delays identified: []
Processing person folder: Jasmine_Edoofa
Filtering messages from the last 7 days in file: C:\Users\mauriceyeng\Python\Daily-Reports\Test\V1_maurice\TestingData\2023-11-18\EWYL TEAM\Jasmine_Edoofa\WhatsApp Chat with Silence EWYL22E1150.txt
Found 0 messages from the last 7 days.
Calculating delays and identifying the person...
Delays identified: []
Processing person folder: Sharda_Edoofa
Filtering messages from the last 7 days in file: C:\Users\mauriceyeng\Python\Dail

In [7]:
import os
import pandas as pd
import datetime
import re

DELAY_THRESHOLD = 15  # minutes

# Function to check if a string is numeric
def is_numeric(s):
    return s.replace('+', '', 1).isdigit()


# Adjusted regex to account for any whitespace character between time and am/pm
regex_pattern = r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s[ap]m -'

# Function to read chat files and extract only the last 7 messages
def read_chat_file(file_path):
    print(f"Reading file: {file_path}")  # Debug
    messages = []
    regex_pattern = r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s[ap]m -'  # Updated regex pattern
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if re.match(regex_pattern, line):
                messages.append(line.strip())
    last_7_messages = messages[-7:]  # Extract only the last 7 messages
    for msg in last_7_messages:  # Debug line to print last 7 messages
        print(f"Extracted message: {msg}")
    print(f"Total messages extracted: {len(last_7_messages)}")  # Debug
    return last_7_messages

# Helper function to extract timestamp and sender from a message
def extract_timestamp_and_sender(message):
    timestamp_str, sender_and_message = message.split(' - ', 1)
    sender = sender_and_message.split(':', 1)[0]
    timestamp = pd.to_datetime(timestamp_str, format='%d/%m/%y, %I:%M %p', errors='coerce')
    return timestamp, sender

# Helper function to calculate time difference in minutes
def calculate_time_difference(start_time, end_time):
    return (end_time - start_time).total_seconds() / 60

# Function to check for delay and calculate delay time
def check_for_delay_and_calculate_time(messages):
    last_7_messages = messages[-7:]
    delay_detected = False
    delay_time = None

    for i in range(len(last_7_messages) - 1):
        current_msg = last_7_messages[i]
        next_msg = last_7_messages[i + 1]
        print(f"Current message: {current_msg}")
        print(f"Next message: {next_msg}")

        current_timestamp, current_sender = extract_timestamp_and_sender(current_msg)
        next_timestamp, next_sender = extract_timestamp_and_sender(next_msg)

        if not is_numeric(current_sender) and is_numeric(next_sender):
            time_diff = calculate_time_difference(current_timestamp, next_timestamp)
            if time_diff > DELAY_THRESHOLD:
                print(f"Time difference: {time_diff} minutes")
                delay_detected = True
                delay_time = time_diff
                break
    
    
    

    return delay_detected, delay_time, ' | '.join(last_7_messages)

# Main analysis process
def main_analysis(root_directory):
    print(f"Starting delay analysis in root directory: {root_directory}")  # Debug
    analysis_table = pd.DataFrame(columns=['Date', 'Chat Group Name', 'Person', 'Delay Detected', 'Delay Time', 'Last 7 Messages'])
    analysis_data = []  # List to store analysis data

    for date_folder in os.listdir(root_directory):
        date_path = os.path.join(root_directory, date_folder)
        print(f"Processing date folder: {date_folder}")  # Debug
        for team_folder in os.listdir(date_path):
            team_path = os.path.join(date_path, team_folder)
            print(f"Processing team folder: {team_folder}")  # Debug
            for person_folder in os.listdir(team_path):
                person_path = os.path.join(team_path, person_folder)
                print(f"Processing person folder: {person_folder}")  # Debug
                for file in os.listdir(person_path):
                    if file.endswith('.txt'):
                        chat_file_path = os.path.join(person_path, file)
                        messages = read_chat_file(chat_file_path)
                        delay_detected, delay_time, last_7_messages = check_for_delay_and_calculate_time(messages)
                        
                        analysis_data.append({
                            'Date': date_folder, 
                            'Chat Group Name': team_folder, 
                            'Person': person_folder, 
                            'Delay Detected': delay_detected, 
                            'Delay Time': delay_time, 
                            'Last 7 Messages': last_7_messages
                        })
                        print(f"Processed file: {file}")  # Debug

    print("Analysis complete. Returning DataFrame.")  # Debug
    return analysis_table

# Run the analysis
root_directory = 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Test\\filtered_chats'  # Replace with the actual path
delay_analysis_table = main_analysis(root_directory)
print(delay_analysis_table.head())  # Displaying the first few rows of the analysis table


Starting delay analysis in root directory: C:\Users\maurice\Documents\Chat-Analyzer-V2\Test\filtered_chats
Processing date folder: 2023-11-28
Processing team folder: EWYL
Processing person folder: Aditi_Edoofa
Processing person folder: Ananya_Edoofa
Processing person folder: Jasmine_Edoofa
Processing person folder: Saloni_Edoofa
Processing person folder: Sharda_Edoofa
Processing team folder: KAM
Processing person folder: Ashi_Edoofa
Processing person folder: Kirti_Edoofa
Processing person folder: Milan_Edoofa
Processing person folder: Shivjeet_Edoofa
Processing person folder: Vilsha_Edoofa
Processing team folder: SALES
Processing person folder: Arshita
Reading file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Test\filtered_chats\2023-11-28\SALES\Arshita\WhatsApp Chat with Abdul_Edoofa(28_11)AA.txt
Extracted message: 27/11/23, 4:59 pm - Arshita: 11/20/2023 11:33:18	Abdulrazak Abdullahi 	abdulrazak2abdul@gmail.com	+234906443753	3/27/2001	Male	Al-ihsan mallawa academy 	Private	Nigeria 	Ab

Extracted message: 20/09/23, 8:42 pm - +263 77 474 0742: Sorry lm not able to make it will be at school attending lessons
Extracted message: 21/09/23, 9:32 am - Arshita: Greetings!
Extracted message: 21/09/23, 4:48 pm - Arshita: I was looking forward to your response. Let us connect at  *9:30 am CAT on Friday(22/09/23) to resume your session (In case you have any challenges or lessons, you can let me know and we will plan accordingly)*
Extracted message: 22/09/23, 9:29 am - Arshita: Greetings!
Extracted message: 22/09/23, 4:30 pm - Arshita: Greetings everyone
Extracted message: 25/11/23, 7:05 am - +263 77 788 4556: Hie
Extracted message: 27/11/23, 11:56 am - Arshita: Greetings!
Total messages extracted: 7
Current message: 20/09/23, 8:42 pm - +263 77 474 0742: Sorry lm not able to make it will be at school attending lessons
Next message: 21/09/23, 9:32 am - Arshita: Greetings!
Current message: 21/09/23, 9:32 am - Arshita: Greetings!
Next message: 21/09/23, 4:48 pm - Arshita: I was looki

Extracted message: 27/11/23, 2:49 pm - Austin#276: We have our Department of Student Welfare at our campuses who will be in touch with students on a regular basis. Moreover, our EWYL team and mentors will be connected to Edoofians to guide them at every stage and assist with the projects. Therefore, any misbehaviour will be addressed by our team, counselling the students and guiding them in the right path. Further misbehaviour would invite the action as per the guidelines of the Universities.
Extracted message: 27/11/23, 2:50 pm - Austin#276: We have a team closely monitoring and guiding Edoofians due to which our students stay career centred and focused on their goals
Extracted message: 27/11/23, 2:50 pm - Austin#276: <Media omitted>
Extracted message: 27/11/23, 2:51 pm - Austin#276: <Media omitted>
Extracted message: 27/11/23, 3:12 pm - +263 77 532 6491: Why I asked is because most of the children they can misbehave due to peer pressure when they are away from parents ruining their f

Extracted message: 24/11/23, 12:39 pm - Pallika Edoofa: Greetings,
Extracted message: 24/11/23, 5:32 pm - +27 67 752 2982: Yes we are good
Extracted message: 24/11/23, 5:33 pm - +27 67 752 2982: Can u pliz reschedule me on the time I’ll be free from 6pm
Extracted message: 24/11/23, 9:56 pm - Pallika Edoofa: <Media omitted>
Extracted message: 27/11/23, 12:33 pm - Pallika Edoofa: Greetings,
Extracted message: 28/11/23, 12:12 am - Pallika Edoofa: Hello Evelyn, I hope you are doing well.
Total messages extracted: 7
Current message: 23/11/23, 9:36 pm - +27 67 752 2982: I would love to proceed with my education but had no fundings. So a friend of mine introduced me to this program and he left for India last month which even motivated me more thus i decided to partake in it and sent my application because i could really see my self embark on this journey with Edoofah. And im super glad I’m here today. It has always been my passion to study abroad due to lack of resources in Zimbabwe so being 

Current message: 24/11/23, 10:03 pm - Pallika Edoofa: <Media omitted>
Next message: 24/11/23, 10:03 pm - +263 71 598 2393: Okay
Current message: 24/11/23, 10:03 pm - +263 71 598 2393: Okay
Next message: 27/11/23, 12:39 pm - Pallika Edoofa: Greetings!
Current message: 27/11/23, 12:39 pm - Pallika Edoofa: Greetings!
Next message: 27/11/23, 12:39 pm - +263 71 598 2393: Yeah sure
Current message: 27/11/23, 12:39 pm - +263 71 598 2393: Yeah sure
Next message: 27/11/23, 1:34 pm - Pallika Edoofa: Kindly excuse me for the delay as there is an issue with my whatsapp today.
Current message: 27/11/23, 1:34 pm - Pallika Edoofa: Kindly excuse me for the delay as there is an issue with my whatsapp today.
Next message: 27/11/23, 1:34 pm - Pallika Edoofa: So, Vimbayi, I would like to know that have you discussed about the program with your family?
Current message: 27/11/23, 1:34 pm - Pallika Edoofa: So, Vimbayi, I would like to know that have you discussed about the program with your family?
Next mess

...

Extracted message: 24/11/23, 5:25 am - +263 77 532 6412: Okay
Extracted message: 24/11/23, 12:02 pm - Sahil Edoofa: Good morning
Extracted message: 24/11/23, 6:07 pm - +263 77 532 6412: I'm fine and how are you
Extracted message: 27/11/23, 12:47 pm - Sahil Edoofa: ??
Extracted message: 27/11/23, 2:24 pm - +263 77 532 6412: 😢😢
Extracted message: 27/11/23, 6:09 pm - Sahil Edoofa: I apologize for the delay today; unfortunately, I'm not feeling well. We'll have to reschedule today's session, and I'll reconnect with you tomorrow at 11:30 PM CAT time
Extracted message: 27/11/23, 9:26 pm - +263 77 532 6412: It's okay
Total messages extracted: 7
Current message: 24/11/23, 5:25 am - +263 77 532 6412: Okay
Next message: 24/11/23, 12:02 pm - Sahil Edoofa: Good morning
Current message: 24/11/23, 12:02 pm - Sahil Edoofa: Good morning
Next message: 24/11/23, 6:07 pm - +263 77 532 6412: I'm fine and how are you
Current message: 24/11/23, 6:07 pm - +263 77 532 6412: I'm fine and how are you
Next messa

Extracted message: 24/11/23, 6:11 pm - +263 77 391 9609: Afternoon Sir we had no electricity the whole day
Extracted message: 24/11/23, 8:47 pm - Sahil Edoofa: Shall I get you the slots for Monday?
Extracted message: 24/11/23, 8:50 pm - +263 77 391 9609: Okay Sir will be available but for the video calls my area is remote
Extracted message: 24/11/23, 8:56 pm - Sahil Edoofa: What time on monday ?
Extracted message: 24/11/23, 9:27 pm - +263 77 391 9609: i will be free at 1900hrs
Extracted message: 26/11/23, 8:25 am - +263 77 391 9609: morning Sir
Extracted message: 27/11/23, 1:15 pm - Sahil Edoofa: Good morning
Total messages extracted: 7
Current message: 24/11/23, 6:11 pm - +263 77 391 9609: Afternoon Sir we had no electricity the whole day
Next message: 24/11/23, 8:47 pm - Sahil Edoofa: Shall I get you the slots for Monday?
Current message: 24/11/23, 8:47 pm - Sahil Edoofa: Shall I get you the slots for Monday?
Next message: 24/11/23, 8:50 pm - +263 77 391 9609: Okay Sir will be availa

Current message: 21/11/23, 11:10 pm - Shashwat Edoofa 2: <Media omitted>
Next message: 22/11/23, 11:33 am - Shashwat Edoofa 2: Good Morning,
Current message: 22/11/23, 11:33 am - Shashwat Edoofa 2: Good Morning,
Next message: 22/11/23, 9:55 pm - Shashwat Edoofa 2: Hello Evelyn, I hope you are doing well.
Current message: 22/11/23, 9:55 pm - Shashwat Edoofa 2: Hello Evelyn, I hope you are doing well.
Next message: 27/11/23, 11:58 am - Shashwat Edoofa 2: Hi Evelyn, I hope you are in good health.
Processed file: WhatsApp Chat with Evelyn Tizirayi Edoofa (06_11) SZ.txt
Reading file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Test\filtered_chats\2023-11-28\SALES\Shashwat Edoofa 2\WhatsApp Chat with Freelance Maseve Edoofa (10_10) SZ.txt
Extracted message: 27/11/23, 7:01 pm - Shashwat Edoofa 2: Can you please acknowledge this question?
Extracted message: 27/11/23, 10:26 pm - +263 71 429 5122: Yeah am ready
Extracted message: 27/11/23, 10:40 pm - Shashwat Edoofa 2: I admire your willingness 

Current message: 23/11/23, 10:37 am - Shashwat Edoofa 2: Good Morning,
Next message: 27/11/23, 11:58 am - Shashwat Edoofa 2: Hi Quenton, I hope you are in good health.
Processed file: WhatsApp Chat with Quenton Mahuni Edoofa (10_11) SZ.txt
Reading file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Test\filtered_chats\2023-11-28\SALES\Shashwat Edoofa 2\WhatsApp Chat with Ritah Langa Kawimbe Edoofa (22_11) SZ.txt
Extracted message: 24/11/23, 10:35 pm - Shashwat Edoofa 2: <Media omitted>
Extracted message: 25/11/23, 5:26 am - +260 96 6555901: Alright. Well noted.
Extracted message: 27/11/23, 11:27 am - Shashwat Edoofa 2: Good Morning,
Extracted message: 27/11/23, 11:31 am - +260 96 6555901: Good morning. My apologies for today. I have an engagement
Extracted message: 27/11/23, 12:12 pm - Shashwat Edoofa 2: I hope you are having a wonderful day, Mr. Alexander!
Extracted message: 27/11/23, 12:12 pm - Shashwat Edoofa 2: Certainly, Mr. Alexander. I will be waiting for your response.
Extracted 

Extracted message: 26/11/23, 10:43 am - Shubham Madhwal: Excuse me for my delayed response I wasn't able to get back to you yesterday because of a family emergency.
Extracted message: 26/11/23, 10:43 am - Shubham Madhwal: I hope you all are doing well please?
Extracted message: 26/11/23, 10:44 am - Shubham Madhwal: This message was deleted
Extracted message: 26/11/23, 10:46 am - +263 78 377 2781: Good morning sir how are you.
Extracted message: 26/11/23, 10:46 am - +263 78 377 2781: Yes we are doing very well thank you
Extracted message: 26/11/23, 10:54 am - Shubham Madhwal: This message was deleted
Extracted message: 27/11/23, 11:11 am - Shubham Madhwal: Are we connected @27737219828 @263783593657 @263718543162 @263783772781?
Total messages extracted: 7
Current message: 26/11/23, 10:43 am - Shubham Madhwal: Excuse me for my delayed response I wasn't able to get back to you yesterday because of a family emergency.
Next message: 26/11/23, 10:43 am - Shubham Madhwal: I hope you all are d

# V2

In [8]:
import os
import pandas as pd
import datetime
import re

DELAY_THRESHOLD = 15  # minutes in delay threshold

# Regex pattern for message start (adjust based on your chat format)
regex_pattern = r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s[ap]m -'

# Function to read chat files and extract messages
def read_chat_file(file_path):
    messages = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if re.match(regex_pattern, line):
                messages.append(line.strip())
    return messages

# Helper function to extract timestamp and sender from a message
def extract_timestamp_and_sender(message):
    timestamp_str, sender_and_message = message.split(' - ', 1)
    sender = sender_and_message.split(':', 1)[0]
    timestamp = pd.to_datetime(timestamp_str, format='%d/%m/%y, %I:%M %p', errors='coerce')
    return timestamp, sender

# Function to aggregate messages by date
def aggregate_messages_by_date(root_directory):
    aggregated_messages = {}
    for date_folder in os.listdir(root_directory):
        date_path = os.path.join(root_directory, date_folder)
        messages_for_date = []
        for team_folder in os.listdir(date_path):
            team_path = os.path.join(date_path, team_folder)
            for person_folder in os.listdir(team_path):
                person_path = os.path.join(team_path, person_folder)
                for file in os.listdir(person_path):
                    if file.endswith('.txt'):
                        chat_file_path = os.path.join(person_path, file)
                        messages = read_chat_file(chat_file_path)
                        messages_for_date.extend(messages)
        aggregated_messages[date_folder] = messages_for_date
    return aggregated_messages

# Function to check for delay and calculate delay time
def check_for_delay_and_calculate_time(messages):
    delays = []
    for i in range(len(messages) - 8):  # Iterate with enough messages left for last 7
        current_msg = messages[i]
        next_msg = messages[i + 1]
        current_timestamp, current_sender = extract_timestamp_and_sender(current_msg)
        next_timestamp, next_sender = extract_timestamp_and_sender(next_msg)

        if current_sender == next_sender:  # Check delay for the same sender
            time_diff = (next_timestamp - current_timestamp).total_seconds() / 60
            if time_diff > DELAY_THRESHOLD:
                delay_info = (i, time_diff)  # Store index and time difference
                delays.append(delay_info)
    
    return delays

# Main analysis process
def main_analysis(root_directory):
    analysis_table = pd.DataFrame(columns=['Date', 'Person', 'Delay Detected', 'Delay Time', 'Last 7 Messages'])
    aggregated_messages = aggregate_messages_by_date(root_directory)
    
    for date, messages in aggregated_messages.items():
        delays = check_for_delay_and_calculate_time(messages)
        for delay_index, delay_time in delays:
            last_7_messages = ' | '.join(messages[delay_index - 6:delay_index + 1])
            sender = extract_timestamp_and_sender(messages[delay_index])[1]
            analysis_table = analysis_table.append({
                'Date': date,
                'Person': sender,
                'Delay Detected': True,
                'Delay Time': delay_time,
                'Last 7 Messages': last_7_messages
            }, ignore_index=True)

    return analysis_table

# Run the analysis
root_directory = 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Test\\filtered_chats'  # Replace with the actual path
delay_analysis_table = main_analysis(root_directory)
print(delay_analysis_table)




            Date   Person Delay Detected  Delay Time  \
0     2023-11-28  Arshita           True       100.0   
1     2023-11-28  Arshita           True       919.0   
2     2023-11-28  Arshita           True       543.0   
3     2023-11-28  Arshita           True      1012.0   
4     2023-11-28  Arshita           True       480.0   
...          ...      ...            ...         ...   
6412  2023-11-28   Tushti           True       793.0   
6413  2023-11-28   Tushti           True       112.0   
6414  2023-11-28   Tushti           True      5582.0   
6415  2023-11-28   Tushti           True       438.0   
6416  2023-11-28   Tushti           True       867.0   

                                        Last 7 Messages  
0                                                        
1     13/11/23, 6:30 pm - Arshita added you | 13/11/...  
2     13/11/23, 6:32 pm - Arshita added +265 984 27 ...  
3     13/11/23, 6:30 pm - Arshita: Michelle EWYL23E0...  
4     13/11/23, 6:32 pm - Arshita: <M

In [9]:
delay_analysis_table.head(5)

,Date,Person,Delay Detected,Delay Time,Last 7 Messages
0,2023-11-28,Arshita,True,100.0,
1,2023-11-28,Arshita,True,919.0,"13/11/23, 6:30 pm - Arshita added you | 13/11/..."
2,2023-11-28,Arshita,True,543.0,"13/11/23, 6:32 pm - Arshita added +265 984 27 ..."
3,2023-11-28,Arshita,True,1012.0,"13/11/23, 6:30 pm - Arshita: Michelle EWYL23E0..."
4,2023-11-28,Arshita,True,480.0,"13/11/23, 6:32 pm - Arshita: <Media omitted> |..."


In [10]:
delay_analysis_table.to_csv("tab3.csv",index=False)